In [ ]:
import pandas as pd
import numpy as np
import re
import string
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

# Download NLTK resources
nltk.download('stopwords')

# Load the dataset
file_path = "olid-training-v1.0.xlsx"
df = pd.read_excel(file_path)

# Mapping OLID labels to toxicity categories
def map_toxicity(row):
    categories = {
        'toxic': 0, 'severe_toxic': 0, 'obscene': 0,
        'threat': 0, 'insult': 0, 'identity_hate': 0
    }

    if row['subtask_a'] == 'OFF':
        categories['toxic'] = 1  # General toxicity
        if row['subtask_b'] == 'TIN':
            categories['insult'] = 1  # Insults
            if row['subtask_c'] == 'IND' or row['subtask_c'] == 'GRP':
                categories['identity_hate'] = 1  # Identity-based hate
            if row['subtask_c'] == 'OTH':
                categories['threat'] = 1  # Other threats
        elif row['subtask_b'] == 'UNT':
            categories['obscene'] = 1  # Untargeted offensive speech
    return pd.Series(categories)

# Apply the mapping
df[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']] = df.apply(map_toxicity, axis=1)

# Text cleaning function
def clean_text(text):
    # Convert to string to handle potential non-string values
    text = str(text)
    text = text.lower()
    text = re.sub(r'@\w+', '', text)  # Remove mentions
    text = re.sub(r'https?://\S+', '', text)  # Remove URLs
    text = re.sub(f"[{string.punctuation}]", '', text)  # Remove punctuation
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = ' '.join(word for word in text.split() if word not in stopwords.words('english'))
    return text

# Apply text cleaning
df['clean_tweet'] = df['tweet'].apply(clean_text)

# Feature extraction using TF-IDF
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(df['clean_tweet'])
y = df[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']]

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize models
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Support Vector Machine": SVC(kernel='linear', probability=True),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss')
}

# Train and evaluate models for each toxicity category
for category in ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']:
    print(f"Training models for {category}...")
    # Select target variable for the current category
    y_train_category = y_train[category]
    y_test_category = y_test[category]

    # Check if y_train_category contains more than one class
    if len(y_train_category.unique()) > 1:
        for name, model in models.items():
            print(f"Training {name}...")
            # Fit the model using the target variable for the current category
            model.fit(X_train, y_train_category)
            y_pred = model.predict(X_test)
            accuracy = accuracy_score(y_test_category, y_pred)
            print(f"Performance of {name} for {category}:")
            print(classification_report(y_test_category, y_pred))
            print(f"Accuracy: {accuracy:.4f}")
            print("-" * 50)
    else:
        print(f"Skipping {category} due to insufficient class representation in training data.")
